In [1]:
import os
import time
import logging
from pathlib import Path
from dataclasses import dataclass

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW, Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, StepLR, OneCycleLR
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import timm

In [2]:
@dataclass
class Config:
    train_csv: str = '/kaggle/input/birdclef-2025/train.csv'
    taxonomy_csv: str = '/kaggle/input/birdclef-2025/taxonomy.csv'
    spectrogram_npy: str = '/kaggle/input/falcon-birdclef-cnn-preprocessed-dataset/falcon_birdclef_cnn_preprocessed_dataset.npy'
    train_datadir: str = '/kaggle/input/birdclef-2025/train_audio'
    LOAD_DATA: bool = True
    n_fold: int = 5
    selected_folds: tuple = (0, 1, 2, 3, 4)
    seed: int = 42
    debug: bool = False
    batch_size: int = 32
    num_workers: int = 4
    epochs: int = 10
    device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    model_name: str = 'efficientnet_b0'
    pretrained: bool = True
    in_channels: int = 1
    optimizer: str = 'AdamW'
    scheduler: str = 'CosineAnnealingLR'
    T_max: int = 10
    min_lr: float = 1e-5
    criterion: str = 'BCEWithLogitsLoss'
    lr: float = 1e-3
    weight_decay: float = 1e-6

cfg = Config()
logging.basicConfig(level=logging.INFO)

In [3]:
def set_seed(seed: int = 42):
    import random
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(cfg.seed)

In [4]:
logging.info("Loading spectrograms...")
spec_path = Path(cfg.spectrogram_npy)
spectrograms = np.load(spec_path, allow_pickle=True).item()
logging.info(f"Loaded {len(spectrograms)} spectrograms")

In [5]:
class SpectrogramDataset(Dataset):
    def __init__(self, df: pd.DataFrame, cfg: Config, specs: dict, mode: str = 'train'):
        self.df = df.copy()
        self.specs = specs
        self.cfg = cfg
        self.mode = mode

        # prepare sample keys
        if 'sample_key' not in self.df.columns:
            self.df['sample_key'] = (
                self.df.filename
                .str.replace('/', '_')
                .str.replace('.wav', '')
            )

        # label mapping
        taxonomy = pd.read_csv(cfg.taxonomy_csv)
        labels = taxonomy['primary_label'].tolist()
        self.label_to_idx = {lbl: idx for idx, lbl in enumerate(labels)}
        self.num_classes = len(labels)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        key = row['sample_key']
        spec = self.specs.get(key)

        if spec is None:
            # fallback: zero tensor
            spec = np.zeros((cfg.in_channels, 256, 256), dtype=np.float32)
        else:
            spec = np.expand_dims(spec, axis=0)  # add channel

        spec = torch.tensor(spec, dtype=torch.float32)

        # encode primary label
        target = np.zeros(self.num_classes, dtype=np.float32)
        primary = row['primary_label']
        if primary in self.label_to_idx:
            target[self.label_to_idx[primary]] = 1.0

        # include secondary if present
        sec = row.get('secondary_labels')
        if isinstance(sec, str) and sec:
            for s in eval(sec):
                if s in self.label_to_idx:
                    target[self.label_to_idx[s]] = 1.0

        target = torch.tensor(target)
        return spec, target

In [6]:
def collate_specs(batch):
    specs, targets = zip(*batch)
    specs = torch.stack(specs)
    targets = torch.stack(targets)
    return specs, targets

In [7]:
class CLEFClassifier(nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        taxonomy = pd.read_csv(cfg.taxonomy_csv)
        num_classes = len(taxonomy)

        back = timm.create_model(
            cfg.model_name,
            pretrained=cfg.pretrained,
            in_chans=cfg.in_channels,
            num_classes=0,
        )
        self.encoder = back
        self.pool = nn.AdaptiveAvgPool2d(1)
        feat_dim = back.num_features
        self.head = nn.Linear(feat_dim, num_classes)

    def forward(self, x):
        feats = self.encoder(x)
        if feats.dim() == 4:
            feats = self.pool(feats)
            feats = feats.view(feats.size(0), -1)
        return self.head(feats)

In [8]:
def make_optimizer(model, cfg):
    if cfg.optimizer == 'AdamW':
        return AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    if cfg.optimizer == 'Adam':
        return Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    if cfg.optimizer == 'SGD':
        return SGD(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay, momentum=0.9)
    raise ValueError('Unknown optimizer')

In [9]:
def make_scheduler(opt, cfg):
    if cfg.scheduler == 'CosineAnnealingLR':
        return CosineAnnealingLR(opt, T_max=cfg.T_max, eta_min=cfg.min_lr)
    if cfg.scheduler == 'ReduceLROnPlateau':
        return ReduceLROnPlateau(opt, factor=0.5, patience=2, min_lr=cfg.min_lr)
    if cfg.scheduler == 'StepLR':
        return StepLR(opt, step_size=cfg.epochs//3, gamma=0.5)
    if cfg.scheduler == 'OneCycleLR':
        return OneCycleLR(opt, max_lr=cfg.lr, steps_per_epoch=1, epochs=cfg.epochs)
    return None

In [10]:
def make_loss(cfg):
    if cfg.criterion == 'BCEWithLogitsLoss':
        return nn.BCEWithLogitsLoss()
    raise ValueError('Unknown loss')

In [11]:
def epoch_step(model, loader, opt, loss_fn, device, scheduler=None, train=True):
    model.train() if train else model.eval()
    losses, all_t, all_p = [], [], []

    loader = tqdm(loader, desc='Train' if train else 'Valid')
    for batch in loader:
        specs, targets = batch
        specs, targets = specs.to(device), targets.to(device)

        if train:
            opt.zero_grad()
            preds = model(specs)
            loss = loss_fn(preds, targets)
            loss.backward()
            opt.step()
        else:
            with torch.no_grad():
                preds = model(specs)
                loss = loss_fn(preds, targets)

        probs = torch.sigmoid(preds).detach().cpu().numpy()
        y = targets.detach().cpu().numpy()
        losses.append(loss.item())
        all_t.append(y)
        all_p.append(probs)
        if scheduler and train:
            if isinstance(scheduler, OneCycleLR):
                scheduler.step()

    all_t = np.vstack(all_t)
    all_p = np.vstack(all_p)
    auc = np.mean([roc_auc_score(all_t[:, i], all_p[:, i]) 
                   for i in range(all_t.shape[1]) if all_t[:, i].sum() > 0])
    return np.mean(losses), auc

In [12]:
def run_cv(df: pd.DataFrame, cfg: Config):
    skf = StratifiedKFold(n_splits=cfg.n_fold, shuffle=True, random_state=cfg.seed)
    scores = []

    for fold, (tr_idx, val_idx) in enumerate(skf.split(df, df['primary_label'])):
        if fold not in cfg.selected_folds:
            continue
        print(f"\n--- Fold {fold} ---")
        tr_df, v_df = df.iloc[tr_idx], df.iloc[val_idx]
        train_ds = SpectrogramDataset(tr_df, cfg, spectrograms, 'train')
        val_ds   = SpectrogramDataset(v_df, cfg, spectrograms, 'valid')
        tr_loader = DataLoader(train_ds, batch_size=cfg.batch_size,
                               shuffle=True, num_workers=cfg.num_workers,
                               collate_fn=collate_specs)
        v_loader  = DataLoader(val_ds,   batch_size=cfg.batch_size,
                               shuffle=False, num_workers=cfg.num_workers,
                               collate_fn=collate_specs)

        model = CLEFClassifier(cfg).to(cfg.device)
        opt = make_optimizer(model, cfg)
        sch = make_scheduler(opt, cfg)
        loss_fn = make_loss(cfg)

        best_auc = 0.0
        for epoch in range(cfg.epochs):
            print(f"Epoch {epoch+1}/{cfg.epochs}")
            train_loss, train_auc = epoch_step(model, tr_loader, opt, loss_fn, cfg.device, sch, True)
            valid_loss, valid_auc = epoch_step(model, v_loader, None, loss_fn, cfg.device, None, False)
            print(f"  Train AUC: {train_auc:.4f}, Valid AUC: {valid_auc:.4f}")

            if valid_auc > best_auc:
                best_auc = valid_auc
                torch.save(model.state_dict(), f"best_fold{fold}.pt")
        scores.append(best_auc)
        print(f"Fold {fold} best AUC: {best_auc:.4f}")

    print(f"\nMean AUC across folds: {np.mean(scores):.4f}")

In [ ]:
if __name__ == '__main__':
    df = pd.read_csv(cfg.train_csv)
    run_cv(df, cfg)

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(



--- Fold 0 ---


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Epoch 1/10


Train:   0%|          | 0/715 [00:00<?, ?it/s]

...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd1e9687a60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd1e9687a60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 15

Valid:   0%|          | 0/179 [00:00<?, ?it/s]

  Train AUC: 0.4577, Valid AUC: 0.5000
Epoch 10/10


Train:   0%|          | 0/715 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd1e9687a60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd1e9687a60>

  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
Traceback (most recent call last):
      File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
if w.is_alive():    
  self._shutdown_workers()  
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
     if w.is_alive():  
^  ^ ^ ^ ^ ^ ^^^^^^^^^
^  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
^    ^^assert self._parent_pid == os.getpid(), 'can only test a child process'^
^ ^  ^^  
   File "/usr/lib/

In [ ]:
# — after run_cv(df, cfg) completes —
# Pick one of the saved fold‐models (e.g. fold 0) and re‐save it as model.pth
best_fold = cfg.selected_folds[0]
best_file = f"best_fold{best_fold}.pt"
print(f"Loading weights from {best_file} and saving as model.pth")

# Instantiate a fresh model, load weights, then dump
final_model = CLEFClassifier(cfg).to(cfg.device)
final_model.load_state_dict(torch.load(best_file, map_location=cfg.device))
torch.save(final_model.state_dict(), 'model.pth')
print("✅ Saved final_model.state_dict() → model.pth")